# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

- **Consider a significance level of 5% for all tests.**

In [7]:
# import numpy and pandas
import numpy as np
import pandas as pd


# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [8]:
# Run this code:

pokemon = pd.read_csv('../pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [9]:
# Your code here:
pokemon.head()



,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [10]:
# Your code here:
pokemon['Legendary'].value_counts()




False    735
True      65
Name: Legendary, dtype: int64

Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [12]:
# Your code here:
pokemon.groupby(['Legendary'])['Total'].agg(['mean','std'])



,mean,std
Legendary,,
False,417.213605,106.760417
True,637.384615,60.937389


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [14]:
# Your code here:
import scipy.stats as st

alpha = 0.05

legendary = pokemon[pokemon['Legendary'] == True]
regular = pokemon[pokemon['Legendary'] == False]

'''
H0: mu total of legendary pokemons = mu total of regular pokemons
H1: mu total of legendary pokemons != mu total of regular pokemons 
'''

stat, p_value = st.ttest_ind(legendary['Total'],regular['Total'], equal_var = False)
print("Statistic", stat)
print("pvalue", p_value)

# Decision:
 
if (p_value < alpha)==True:
    print("reject the H0")
else:
    print("we can't reject the H0")



Statistic 25.8335743895517
pvalue 9.357954335957444e-47
reject the H0


What do you conclude from this test? Write your conclusions below.

In [ ]:
# Your conclusions here:
'''the legendary pokemon and the regular pokemon don't have the same sample mean'''


How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [15]:
# Your code here:
pokemon['Type 1'].value_counts()


Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Electric     44
Rock         44
Dragon       32
Ground       32
Ghost        32
Dark         31
Poison       28
Steel        27
Fighting     27
Ice          24
Fairy        17
Flying        4
Name: Type 1, dtype: int64

Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [16]:
# Your code here:
pokemon.groupby(pokemon["Type 1"]=="Water")["Total"].agg(['mean','std'])


,mean,std
Type 1,,
False,435.859012,121.091682
True,430.455357,113.188266


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [19]:
# Your code here:

import scipy.stats as st

alpha = 0.05

water_type = pokemon[pokemon['Type 1'] == 'Water']
other_type = pokemon[pokemon['Type 1'] != 'Water']

'''
H0: mu total of water pokemons = mu total of not water pokemons
H1: mu total of water pokemons != mu total of not water pokemons
'''

stat, p_value = st.ttest_ind(water_type['Total'],other_type['Total'], equal_var = False)
print("Statistic", stat)
print("pvalue", p_value)

# Decision:
 
if (p_value < alpha)==True:
    print("reject the H0")
else:
    print("we can't reject the H0")


Statistic -0.4638681676327303
pvalue 0.6433915385821448
we can't reject the H0


Write your conclusion below.

In [ ]:
# Your conclusions here:
''' the water type total and the other types total have the same mean with  a 95% significant level'''


# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [22]:
# Your code here:
import scipy.stats as st
## 1 step:
''' H0: mu defence score = mu attack score
    H1:mu defence score =! mu attack score
'''

alpha=0.05

## 3 step: sample (we are using the same sample as above)

''' we have the sample'''
pokemon

## 4 step: comput the test statistic ##5 step: compute the p-value

stat, p_value=st.ttest_rel(pokemon['Attack'],pokemon['Defense']) ## two tailed test
print("Statistic", stat)
print("pvalue", p_value)

# Decision:
 
if (p_value < alpha)==True:
    print("reject the H0")
else:
    print("we can't reject the H0")


Statistic 4.325566393330478
pvalue 1.7140303479358558e-05
reject the H0


Describe the results of the test in the cell below.

In [ ]:
# Your conclusions here:
''' the mean of Attack scores is not significantly equal to the mean of the Defense scores'''


We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [23]:
# Your code here:

## 1 step:
''' H0: mu special defence score = mu special attack score
    H1:mu special defence score =! mu special attack score
'''

alpha=0.05

## 3 step: sample (we are using the same sample as above)

''' we have the sample'''
pokemon

## 4 step: comput the test statistic ##5 step: compute the p-value

stat, p_value=st.ttest_rel(pokemon['Sp. Atk'],pokemon['Sp. Def']) ## two tailed test
print("Statistic", stat)
print("pvalue", p_value)

# Decision:
 
if (p_value < alpha)==True:
    print("reject the H0")
else:
    print("we can't reject the H0")

Statistic 0.853986188453353
pvalue 0.3933685997548122
we can't reject the H0


Describe the results of the test in the cell below.

In [ ]:
# Your conclusions here:
''' the mean of special Attack scores is  significantly equal to the mean of the special Defense scores'''


As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [26]:
# Your code here:
''' H0: mu defference between Attack & defence score = 0
    H1: mu defference between Attack & defence score =! 0
'''

stat, p_value = st.ttest_1samp(pokemon['Attack']-pokemon["Defense"],0)
print("Statistic", stat)
print("pvalue", p_value)

# Decision:
 
if (p_value < alpha)==True:
    print("reject the H0")
else:
    print("we can't reject the H0")
    

Statistic 4.325566393330478
pvalue 1.7140303479358558e-05
reject the H0


# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [ ]:
# Your code here:



Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [ ]:
# Your code here:



Based on a 95% confidence, should we reject the null hypothesis?

In [ ]:
# Your answer here:

